In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

# Load data : reading the images and converting all to array

In [2]:
Datadirectory = "Data/"
Classes = ["Face_Mask", "No_Mask"]
img_size = 224
trainning_data = []

def create_training_data():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category) # 0 1 # Label
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path,img)) #load image
                new_arr = cv2.resize(img_arr, (img_size, img_size)) #resize image
                trainning_data.append([new_arr,class_num])
            except Exception as e:
                pass

In [3]:
create_training_data()

## Merge, label, reshape data

In [4]:
# Merge data
import random
random.shuffle(trainning_data)

In [5]:
#label data
X = [] #data/features
y = [] #label

for features, label in trainning_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1,img_size,img_size,3)

In [6]:
X.shape

(1878, 224, 224, 3)

In [7]:
#Normalize the data
X = X/255.0

In [8]:
y = np.array(y)

In [9]:
print(y)

[0 1 1 ... 1 0 0]


## Save data

In [10]:
import pickle

pickle_out = open('X.pickle', 'wb')
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open('y.pickle', 'wb')
pickle.dump(y, pickle_out)
pickle_out.close()

## Read data

In [21]:
pickle_in = open("X.pickle", 'rb')
X = pickle.load(pickle_in)

pickle_in = open('y.pickle', 'rb')
y = pickle.load(pickle_in)

# Deep learning model for training 

## Transfer Learning

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
model = tf.keras.applications.mobilenet.MobileNet()

2022-04-24 22:01:54.087086: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-24 22:01:54.087473: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


In [13]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [14]:
base_input = model.layers[0].input

In [15]:
base_output = model.layers[-4].output

In [16]:
Flat_layer = layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer)
#final_output = layers.Activation('sigmoid')(final_output)

In [17]:
new_model = keras.Model(inputs = base_input, outputs = final_output)

In [18]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

## Setting for binary classification (Face mask/ without mask)

In [19]:
new_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
new_model.fit(X, y, epochs=50, validation_split=0.1, batch_size=64)

Epoch 1/50
27/27 [==============================] - 11s 331ms/step - loss: 0.9586 - accuracy: 0.9337 - val_loss: 0.7384 - val_accuracy: 0.9521
Epoch 2/50
27/27 [==============================] - 9s 324ms/step - loss: 0.5930 - accuracy: 0.9615 - val_loss: 0.7417 - val_accuracy: 0.9468
Epoch 3/50
27/27 [==============================] - 9s 323ms/step - loss: 0.6202 - accuracy: 0.9598 - val_loss: 0.8195 - val_accuracy: 0.9468
Epoch 4/50
27/27 [==============================] - 9s 323ms/step - loss: 0.4939 - accuracy: 0.9669 - val_loss: 0.6554 - val_accuracy: 0.9574
Epoch 5/50
27/27 [==============================] - 9s 323ms/step - loss: 0.4207 - accuracy: 0.9722 - val_loss: 0.5773 - val_accuracy: 0.9628
Epoch 6/50
27/27 [==============================] - 9s 324ms/step - loss: 2.0597 - accuracy: 0.8639 - val_loss: 5.2452 - val_accuracy: 0.5851
Epoch 7/50
27/27 [==============================] - 9s 323ms/step - loss: 1.6753 - accuracy: 0.8882 - val_loss: 1.3932 - val_accuracy: 0.9096
Epoch

In [28]:
new_model.save('face_mask.h5')